In [20]:
# PyTorchライブラリの読み込み
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# tensorflowライブラリの読み込み
import tensorflow.examples.tutorials.mnist.input_data as input_data

# sklearnライブラリの読み込み
from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

# pandas
import pandas as pd

# matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

In [22]:
mnist = datasets.fetch_mldata('MNIST original', data_home='./data')

In [23]:
mnist

{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([ 0.,  0.,  0., ...,  9.,  9.,  9.])}